In [ ]:
import json
import numpy as np
from numpy import dot
from numpy.linalg import norm

file = open('/Users/mango/Documents/GitHub/IN4325/data/STR/queryVectors.json', 'r')
queryVectors = json.load(file)
file.close()

In [ ]:
list(queryVectors[0]['vectors'][1].values())

In [ ]:
file = open('/Users/mango/Documents/GitHub/IN4325/data/STR/queryWordsTFIDF.json', 'r')
queryTFIDFs = json.load(file)
file.close()

In [ ]:
term = queryTFIDFs[0]['words'][0]
queryTFIDFs[0]['termValues'][term]['TFIDF'] * np.array(list(queryVectors[0]['vectors'][0].values()))
queryTFIDFs[0]['id']

In [ ]:
queryVectorCenters = [] #{'queryId':"1", 'center':[[]]}
for j in range(len(queryTFIDFs)):
    q = queryTFIDFs[j]
    v = []
    
    for i in range(len(q['termValues'])):
        term = q['words'][i]
        
        if "dict_values(['not available'])" == str(queryVectors[j]['vectors'][i].values()):
            continue
            
        if (len(v) == 0):
            v = q['termValues'][term]['TFIDF'] * np.array(list(queryVectors[j]['vectors'][i].values()))
            continue
            
        v = np.add(v, q['termValues'][term]['TFIDF'] * np.array(list(queryVectors[j]['vectors'][i].values())))
        
    if len(queryVectorCenters) == 0:
        queryVectorCenters = [{'id': q['id'], 'center': v}]
    else:
        queryVectorCenters.append( {'id': q['id'], 'center': v} )
    

In [ ]:
# queryVectorCenters[1]['center']
# queryVectors[1]['vectors']
queryTFIDFs[1]

In [ ]:
file = open('/Users/mango/Documents/GitHub/IN4325/data/STR/tableVectors.json', 'r')
tableVectors = json.load(file)
file.close()

file = open('/Users/mango/Documents/GitHub/IN4325/data/STR/tableWordsTFIDF.json', 'r')
tableTFIDFs = json.load(file)
file.close()

In [ ]:
len(tableVectors[0]['vectors'])

In [ ]:
term = tableTFIDFs[0]['words'][0]
tableTFIDFs[0]['termValues'][term]

In [ ]:
tableVectorCenters = [] # {'id':'table-1590-536', 'query'Id:'31', 'center':[[]]}
early_fusion = []

# columns in output file
queryIds = []
tableIds = []
earlyFusions = []

for j in range(len(tableVectors)):
    t_v = tableVectors[j]
    t_t = tableTFIDFs[j]
    v = []
    
    for i in range(len(t_v['vectors'])):
        term = t_t['words'][i]
        
        if "dict_values(['not available'])" == str(t_v['vectors'][i].values()):
            continue
            
        if len(v) == 0:
            v = t_t['termValues'][term]['TFIDF'] * np.array(list(t_v['vectors'][i].values()))
            continue
            
        v = np.add(v, t_t['termValues'][term]['TFIDF'] * np.array(list(t_v['vectors'][i].values())) )
        
    current_query_id = int(t_v['query'])
    v_center_q = queryVectorCenters[current_query_id-1]['center']
    if ( type(v) == type(np.array(0)) ) & ( type(v_center_q) == type(np.array(0)) ):
        v_center_t = v.T
        cos_sim = dot(v_center_q, v_center_t)/(norm(v_center_q)*norm(v_center_t)) #Early Fusion: cos (C_q, C_T)
        #cos_sim is a 2-D array with length 1
    else:
        cos_sim = [[0]]

    if len(tableVectorCenters) == 0:
        tableVectorCenters = [{'id': t_v['id'], 'queryId': t_v['query'], 'center': v}]
    else:
        tableVectorCenters.append({'id': t_v['id'], 'queryId': t_v['query'], 'center': v})
        
    if len(early_fusion) == 0:
        early_fusion = [ {'QueryID': t_v['query'], 'Table id': t_v['id'], 'early_fusion': cos_sim[0][0]} ]
    else:
        early_fusion.append( {'QueryID': t_v['query'], 'Table id': t_v['id'], 'early_fusion': cos_sim[0][0]} )
        
    queryIds.append(int(t_v['query']))
    tableIds.append(t_v['id'])
    earlyFusions.append(cos_sim[0][0])
    tableTFIDFs.append()
    

In [ ]:
import math

def checkValueEmpty(list):
    for i in range(len(list)):
        if math.isnan(list[i]):
            print(i, list[i])
            list[i] = 0.000001
            
# checkValueEmpty(queryIds)
# checkValueEmpty(tableIds)
checkValueEmpty(earlyFusions)

In [ ]:
checkValueEmpty(earlyFusions)

In [ ]:
# write to csv file
import pandas as pd

df = pd.DataFrame(data={"QueryID": queryIds, "Table id":tableIds, "early_fusion":earlyFusions})

str_ = df.sort_values(by=['QueryID', 'early_fusion'], ascending = [True, False]).copy()

In [ ]:
str_.to_csv(r'/Users/mango/Desktop/STR.csv', index = False, sep=',')

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum()/df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)
# 4.9% missing data in early_fusion

In [ ]:
str_['rank'] = str_.groupby('QueryID')[['QueryID', 'early_fusion']].rank(ascending=False, method='first')

# Add two more columns to fit the input format of trec_eval tool
str_["query"] = "Q0"
str_["table"] = "smarttable"

# Re-order the columns
str_ = str_[['QueryID', 'query', 'Table id', 'rank', 'early_fusion', 'table']]

# We take the first 5, 10, 15, 20 rankings
str_5 = str_[str_["rank"] < 6]
str_10 = str_[str_["rank"] < 11]
str_15 = str_[str_["rank"] < 16]
str_20 = str_[str_["rank"] < 21]

# We write all rankings into files
str_5.to_csv(r'/Users/mango/Desktop/STR_5.csv', header=None, index=None, sep=',')
str_10.to_csv(r'/Users/mango/Desktop/STR_10.csv', header=None, index=None, sep=',')
str_15.to_csv(r'/Users/mango/Desktop/STR_15.csv', header=None, index=None, sep=',')
str_20.to_csv(r'/Users/mango/Desktop/STR_20.csv', header=None, index=None, sep=',')

In [ ]:
str_.to_csv(r'/Users/mango/Desktop/STR.csv', index=None, sep=',')